In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def load_dataset(dataset_dir, image_size=(64, 64)):
    processed_images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_map = {class_name: i for i, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_folder = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_folder):
            class_images = []
            for filename in os.listdir(class_folder):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(class_folder, filename)
                    image = cv2.imread(image_path)
                    if image is not None:
                        image = cv2.resize(image, image_size)
                        class_images.append(image)
                        labels.append(class_map[class_name])
            if class_images:
                processed_images.extend(class_images)
        else:
            print(f"Warning: '{class_folder}' is not a valid directory.")
    
    if not processed_images:
        raise ValueError("No valid images found in the dataset.")
    
    processed_images = np.array(processed_images)
    labels = np.array(labels)
    
    return processed_images, labels, class_map

    return processed_images, labels, class_map

# Define the path to your dataset directory
dataset_directory = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train"

# Load and preprocess the dataset
images, labels, class_map = load_dataset(dataset_directory)

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [2]:
from tensorflow.keras import layers, models

def build_lnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Output layer for classification
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Get input shape and number of classes from the dataset
input_shape = train_images[0].shape
num_classes = len(class_map)

# Build the LNN model
model = build_lnn_model(input_shape, num_classes)


In [3]:
# Example usage for making predictions
def preprocess_image(image_path, image_size=(64, 64)):
    image = cv2.imread(image_path)
    if image is not None:
        image = cv2.resize(image, image_size)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# # Example of how to use the trained model for prediction
# new_image_path = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Test\seborrheic keratosis\ISIC_0010809.jpg"
# new_image = preprocess_image(new_image_path)
# predictions = model.predict(new_image)
# predicted_class = np.argmax(predictions)
# predicted_class_name = [k for k, v in class_map.items() if v == predicted_class][0]

# print('Predicted class:', predicted_class_name)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Example of using data augmentation during training
train_datagen = datagen.flow(train_images, train_labels, batch_size=32)


In [5]:
from tensorflow.keras.applications import DenseNet121

# Load pre-trained DenseNet model (excluding top layers)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add top layers for classification
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

# Create the complete model
model = models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with data augmentation
model.fit(train_datagen, epochs=10, validation_data=(val_images, val_labels))


Epoch 1/10
56/56 [==============================] - 55s 656ms/step - loss: 1.9160 - accuracy: 0.3579 - val_loss: 25.9224 - val_accuracy: 0.1540
Epoch 2/10
56/56 [==============================] - 31s 553ms/step - loss: 1.5143 - accuracy: 0.4668 - val_loss: 21.1915 - val_accuracy: 0.1786
Epoch 3/10
56/56 [==============================] - 31s 549ms/step - loss: 1.4222 - accuracy: 0.4980 - val_loss: 20.5163 - val_accuracy: 0.1875
Epoch 4/10
56/56 [==============================] - 29s 510ms/step - loss: 1.3434 - accuracy: 0.5260 - val_loss: 20.9349 - val_accuracy: 0.1496
Epoch 5/10
56/56 [==============================] - 18s 319ms/step - loss: 1.2813 - accuracy: 0.5416 - val_loss: 30.2309 - val_accuracy: 0.1674
Epoch 6/10
56/56 [==============================] - 19s 335ms/step - loss: 1.2841 - accuracy: 0.5394 - val_loss: 27.0851 - val_accuracy: 0.1652
Epoch 7/10
56/56 [==============================] - 16s 286ms/step - loss: 1.2696 - accuracy: 0.5438 - val_loss: 41.9422 - val_accuracy:

In [6]:
import numpy as np

# Compute class frequencies
class_counts = np.bincount(train_labels)
total_samples = np.sum(class_counts)

# Compute class weights
class_weights = total_samples / (len(class_counts) * class_counts)

# Convert class weights to dictionary format
class_weight_dict = dict(zip(np.unique(train_labels), class_weights))


In [9]:
# Train multiple models with different hyperparameters or architectures
model1 = build_lnn_model(input_shape, num_classes)
model2 = build_lnn_model(input_shape, num_classes)
# Train model1 and model2 using different training strategies

# Make predictions using each model
predictions1 = model1.predict(val_images)
predictions2 = model2.predict(val_images)

# Combine predictions using voting or averaging
combined_predictions = (predictions1 + predictions2) / 2  # Example of averaging

# Evaluate the combined predictions
combined_accuracy = np.mean(np.argmax(combined_predictions, axis=1) == val_labels)
print('Combined Model Accuracy:', combined_accuracy)


14/14 [==============================] - 0s 16ms/step
Combined Model Accuracy: 0.20758928571428573


In [10]:
import cv2
import numpy as np

def preprocess_image(image_path, image_size=(64, 64)):
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to load image from '{image_path}'.")
        return None
    
    # Resize the image to the specified size
    image = cv2.resize(image, image_size)
    
    # Normalize pixel values (optional, depends on model input)
    image = image / 255.0  # Assuming pixel values range from 0 to 255
    
    # Add batch dimension to match model input shape
    image = np.expand_dims(image, axis=0)
    
    return image

def predict_image(model, image_path, class_map):
    # Preprocess the image
    image = preprocess_image(image_path)
    if image is None:
        return None
    
    # Make predictions using the model
    predictions = model.predict(image)
    
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions[0])
    
    # Get the predicted class name from the class map
    predicted_class_name = [k for k, v in class_map.items() if v == predicted_class_index][0]
    
    return predicted_class_name

# Example usage
image_path = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train\actinic keratosis\ISIC_0025780.jpg" # Specify the path to your image file
predicted_class = predict_image(model, image_path, class_map)

if predicted_class is not None:
    print(f"Predicted class': {predicted_class}")


1/1 [==============================] - 2s 2s/step
Predicted class': actinic keratosis


In [12]:
from tensorflow.keras.models import save_model

# Assuming 'model' is your trained skin cancer classification model
model.save('lnn_model.h5')
